# 概要
ここでは、GoogleColaboratoryの
とGoogleDriveを使用して、BERTを用いた日本語文章の多値分類を行います。  
日本語学習済みモデルは 京都大学 黒橋・河原研究所が公開している`BERT日本語Pretrainedモデル`を、形態素解析器は同研究室が公開している`JUMAN`を使用します。  
  
GoogleDriveに学習済みモデルとデータセットを保存するため、1.6GB以上の空きが必要です。

# 全体の流れ
1. 形態素解析器のインストール
1. GoogleDriveをマウント
1. データセット/BERT日本語学習済みモデルをGoogleDriveへ保存
1. データセットをBERT向けのフォーマットに変換
1. BERTのリポジトリをClone
1. プログラムの改変
1. trainデータを使用した、学習済みモデルのfine-tuning
1. テストデータの予測
1. 予測結果の検証

## 形態素解析器のインストール
京都大学 黒橋・河原研究所が公開している`BERT日本語Pretrainedモデル`では、形態素解析器に`JUMAN`を使用する必要がある為、インストールを行います。  
詳細については、以下をご確認ください。  
[BERT日本語Pretrainedモデル - KUROHASHI-KAWAHARA LAB - 詳細](http://nlp.ist.i.kyoto-u.ac.jp/index.php?BERT日本語Pretrainedモデル#r6199008)

In [ ]:
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc2/jumanpp-2.0.0-rc2.tar.xz && \
tar xJvf jumanpp-2.0.0-rc2.tar.xz && \
rm jumanpp-2.0.0-rc2.tar.xz && \
cd jumanpp-2.0.0-rc2/ && \
mkdir bld && \
cd bld && \
cmake .. \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local && \
make && \
sudo make install

インストールの確認

In [ ]:
!jumanpp -v

### TensorFlowのバージョン変更

In [ ]:
%tensorflow_version 1.x

### pyknpのインストール
PythonでJUMANを実行する為のライブラリです。

In [ ]:
! pip install pyknp

## GoogleDriveのマウント

In [ ]:
from google.colab import drive 

drive.mount('/content/drive')

作業ディレクトリの作成と移動

In [ ]:
!mkdir -p /content/drive/'My Drive'/brrt/livedoor_news

In [ ]:
cd /content/drive/'My Drive'/bert/livedoor_news

/content/drive/My Drive/bert/livedoor_news


## データセット/BERT日本語学習済みモデルをGoogleDriveへ保存

### livedoor newsコーパスのダウンロード

In [ ]:
import urllib.request

livedoor_news_url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
urllib.request.urlretrieve(livedoor_news_url, "ldcc-20140209.tar.gz")

### BERT日本語Pretrainedモデルのダウンロードと解凍

In [ ]:
kyoto_u_bert_url = "http://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JapaneseBertPretrainedModel/Japanese_L-12_H-768_A-12_E-30_BPE.zip"
urllib.request.urlretrieve(kyoto_u_bert_url, "Japanese_L-12_H-768_A-12_E-30_BPE.zip")

In [ ]:
!unzip Japanese_L-12_H-768_A-12_E-30_BPE.zip

## データセットをBERT向けのフォーマットに変換

### TSVファイルの作成
下記の記事を参考に作成しました。  
[BERT多言語モデルで日本語文章の二値分類を試す](https://qiita.com/knok/items/9e3b4505d6b8f813943d)

  GoogleDriveに保存されるまで少し時間がかかります。

In [ ]:
import tarfile
import csv
import re


target_genre = [
                "dokujo-tsushin",
                "it-life-hack",
                "kaden-channel",
                "livedoor-homme",
                "movie-enter",
                "peachy",
                "smax",
                "sports-watch",
                "topic-news"
                ]

fname_list = [[] for i in range(len(target_genre))]

tsv_fname = "all.tsv"

brackets_tail = re.compile('【[^】]*】$')
brackets_head = re.compile('^【[^】]*】')

def remove_brackets(inp):
    output = re.sub(brackets_head, '',re.sub(brackets_tail, '', inp))

    return output

def read_title(f):
    next(f)
    next(f)
    title = next(f)
    title = remove_brackets(title.decode('utf-8'))

    return title[:-1]

with tarfile.open("ldcc-20140209.tar.gz") as tf:
    for ti in tf:
        if "LICENSE.txt" in ti.name:
            continue
        elif "CHANGES.txt" in ti.name:
            continue
        elif "README.txt" in ti.name:
            continue
        else:
            for i, t in enumerate(target_genre):
                if target_genre[i] in ti.name and ti.name.endswith(".txt"):
                    fname_list[i].append(ti.name)
                    continue

    with open(tsv_fname, "w") as wf:
        writer = csv.writer(wf, delimiter='\t')
        for i, fcategory in enumerate(fname_list):
            for name in fcategory:
                f = tf.extractfile(name)
                title = read_title(f)
                row = [target_genre[i], i, '', title]
                writer.writerow(row)

### trainデータ/devデータ/testデータの分割  
all.tsvがGoogleDriveに生成されてから実行

In [ ]:
import random

random.seed(100)
with open("all.tsv", 'r') as f, open("rand-all.tsv", "w") as wf:
    lines = f.readlines()
    random.shuffle(lines)
    for line in lines:
        wf.write(line)

random.seed(101)

train_fname, dev_fname, test_fname = ["train.tsv", "dev.tsv", "test.tsv"]

with open("rand-all.tsv") as f, open(train_fname, "w") as tf, open(dev_fname, "w") as df, open(test_fname, "w") as ef:
    ef.write("class\tsentence\n")
    for line in f:
        v = random.randint(0, 9)
        if v == 8:
            df.write(line)
        elif v == 9:
            ef.write(line)
        else:
            tf.write(line)

## BERTのリポジトリをClone

In [ ]:
!git clone https://github.com/google-research/bert.git

## プログラムの改変
[Qiitaの記事](https://qiita.com/Yuu94/items/0e5cff226bd3cc8fb08c)を参考に、`run_classifier.py`と`tokenization.py`を改変してください。

## trainデータを使用した、学習済みモデルのfine-tuning

In [ ]:
!python bert/run_classifier_livedoor.py \
--task_name=livedoor \
--do_train=true \
--do_eval=true \
--data_dir=./ \
--vocab_file=./Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt \
--bert_config_file=./Japanese_L-12_H-768_A-12_E-30_BPE/bert_config.json \
--init_checkpoint=./Japanese_L-12_H-768_A-12_E-30_BPE/bert_model.ckpt \
--max_seq_length=128 \
--train_batch_size=32 \
--learning_rate=2e-5 \
--num_train_epochs=3.0 \
--output_dir=./tmp/livedoor_news_output_fine \
--do_lower_case False

## テストデータの予測

In [ ]:
!python bert/run_classifier_livedoor.py \
  --task_name=livedoor \
  --do_predict=true \
  --data_dir=./ \
  --vocab_file=./Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt \
  --bert_config_file=./Japanese_L-12_H-768_A-12_E-30_BPE/bert_config.json \
  --init_checkpoint=./tmp/livedoor_news_output_fine \
  --max_seq_length=128 \
  --output_dir=tmp/livedoor_news_output_predic/

## 予測結果の検証

In [ ]:
import csv
import numpy as np


with open("./test.tsv") as f, open("tmp/livedoor_news_output_predic/test_results.tsv") as rf:
  test = csv.reader(f, delimiter = '\t')
  test_result = csv.reader(rf, delimiter = '\t')

  # 正解データの抽出
  next(test)
  test_list = [int(row[1]) for row in test ]

  # 予測結果を抽出
  result_list = []
  for result in test_result:
    max_index = np.argmax(result)
    result_list.append(max_index)

  # 分類した予測結果(カテゴリNo)を出力
  with open('tmp/livedoor_news_output_predic/test_results.csv', 'w') as of:
    writer = csv.writer(of)
    for row in result_list:
      writer.writerow([row])

  test_count = len(test_list)
  result_correct_answer_list = [result for test, result in zip(test_list, result_list) if test == result]
  result_correct_answer_count = len(result_correct_answer_list)
  print("正解率: ", result_correct_answer_count / test_count)
